# Exploration of the experiments

Analyses we care about:
* Plots for all the different experiments


* proportion concluded with a deal
* final payoff per side
* how does starting affect results


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import glob

import pandas as pd 
import numpy as np
import hydra
from omegaconf import DictConfig, OmegaConf
import yaml
from dataclasses import dataclass, field

import matplotlib.pyplot as plt
from hydra import initialize, compose

from omegaconf import OmegaConf, open_dict
from hydra.core.global_hydra import GlobalHydra
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from src.utils import unpack_nested_yaml, load_hydra_config

import omegaconf
from typing import List

## Move this class to a better location alter. 
For now it lets us open a config, import the negotiations, and begin looking at results.

In [4]:
@dataclass
class NegotiationHistory:
    """
    Load and filter a completed negotiation run.
    
    :param history_path:
    """
    history_path: str
    meets_criteria: bool = False
    run_filters: dict = field(default_factory=list)
        
    def __post_init__(self):
        try:
            self.cfg = load_hydra_config(os.path.join("..", self.history_path, ".hydra/"))
        except:
            self.cfg = None
        
        if (len(self.run_filters)>0) and (self.cfg is not None):
            self.meets_criteria = self.search_list(self.run_filters)
        
        if (self.meets_criteria) and (self.cfg is not None):
            try:
                self.df = pd.read_csv(os.path.join(self.history_path, "processed_negotiation.csv"))
            except FileNotFoundError:
                print(f"Couldn't find file '{os.path.join(self.history_path, 'processed_negotiation.csv')}'")

            # load in some of the key data

            self.agent_1 = self.cfg["experiments"]["agent_1"]
            self.agent_2 = self.cfg["experiments"]["agent_2"]
            self.agent_start_id = self.cfg["experiments"]["negotiation_protocol"]["start_agent_index"]
        
    def search_list(self, run_restriction: List[dict]):
        config = self.cfg["experiments"]
        contains_restrictions = []
        for key_val in run_restriction:
            for key, val in key_val.items():
                if key[0] == "~":
                    key = key[1:]
                    contains_restrictions.append(not search_config(config, key, val))
                else:
                    contains_restrictions.append(search_config(config, key, val))
        if all(contains_restrictions):
            return True
        return False
            
    @staticmethod
    def search_config(config, key, val, accu=0):
        def search(config, key, val, accu):
            if hasattr(config, "items"):
                for k, v in config.items():
                    if (k==key) & (v==val):
                        accu += 1
                    elif isinstance(v, omegaconf.dictconfig.DictConfig):
                        accu += search(v, key, val, accu)
                    elif isinstance(v, list):
                        for d in v:
                            accu += search(v, key, val, accu)
            return accu
        accu = search(config, key, val, 0)
        if accu > 0:
            return True
        return False

        
    def create_name(self):
        return f"""Agent {self.agent_start_id} starts negotiations
Agent 0: {self.agent_1.external_description.name} vs. Agent 1: {self.agent_2.external_description.name})
{self.agent_1.msg_max_len} (max msg len) | {self.agent_2.msg_max_len} (max msg len)
{self.agent_1.note_max_len} (max note len) | {self.agent_2.note_max_len} (max note len)
{self.agent_1.model_name} | {self.agent_2.model_name}
"""

In [5]:
runs = glob.glob("logs/inference/runs/*")

In [7]:
log_of_histories = []
restrictions = [
    {'external_description': {'name': 'Representative'}, 
     "temperature": 0, 
     "max_rounds": 15, 
     "start_agent_index": 0,
    }
]
verbose = False

i = 0
for i, run in enumerate(runs):
    history = NegotiationHistory(run, restrictions)
    if not history.meets_criteria:
        continue
    if hasattr(history, "df"):
        print(f"Run: {i}")
        i+=1
        print(f"Number rounds: {max(history.df['round'])}")
        print("Final payoff for agent 0")
        log_of_histories.append(history)
        
        
        if verbose:
            for idx, row in history.df.iloc[-2:].iterrows():
                print(f"Agent id: {row['agent_id']}")
                print(f"Note: {row['note']}")
                print(f"\nMessage: {row['message']}")

                print()

        print("============================================================")
#     groups = history.df.groupby("agent_id")
#     fig, ax = plt.subplots()

#     for name, group in groups:
#         x = group["round"]
#         y = group["normalized_payoff"]
        
#         ax.plot(x,y, label=name, alpha=0.5)
#     plt.title(run_name)
#     plt.legend()
#     plt.show()



Run: 0
Number rounds: 5
Agent id: 1
Note: Mental Note:
The representative of Rio Copa Foods is still insisting on a 3-year non-compete period. We need to maintain our stance on a longer period, but also show flexibility. A 4-year period could be a reasonable compromise, still providing a significant payoff for CPC.

Acceptable Offer:
```json
{
    "non-compete period": "4 years"
}```

Message: We understand your concerns and the importance of your family business. However, considering the strategic importance of this acquisition for CPC, we propose a non-compete period of 4 years. This would ensure a smooth transition and secure our position in the Santa Cruz market. We believe this is a fair compromise that respects the value and contributions of Rio Copa Foods.

Agent id: 0
Note: Mental Note: 

The representative's latest offer aligns with my acceptable compromise. It's important to maintain the value of Rio Copa Foods and secure a good deal for the family. I should accept this offer

Couldn't find file 'logs/inference/runs/2023-08-15_15-11-10/processed_negotiation.csv'
Run: 13
Number rounds: 5
Agent id: 1
Note: Mental Note: Counter with 4 years, final acceptable offer. 

```json
{
    "non-compete period": "4 years"
}
```

Message: Thank you for your proposal. We appreciate your ongoing flexibility. However, we believe a 5-year non-compete period is still quite lengthy. We propose a non-compete period of 4 years, which we believe is a fair compromise. This is our final offer and we believe it strikes a balance between our strategic interests. We look forward to your response.

Agent id: 0
Note: Mental Note: CPC's final offer is 4 years. Counter with 5 years, but be prepared to accept 4 years as a compromise.

```json
{
    "non-compete period": "4 years"
}```

Message: Thank you for your final offer. We appreciate your understanding of our position and your commitment to a fair compromise. We initially proposed a 5-year non-compete period, but in the spirit of reac

Couldn't find file 'logs/inference/runs/2023-08-15_11-00-28/processed_negotiation.csv'
Couldn't find file 'logs/inference/runs/2023-08-15_11-29-35/processed_negotiation.csv'
Couldn't find file 'logs/inference/runs/2023-08-15_15-10-50/processed_negotiation.csv'
Run: 25
Number rounds: 7
Agent id: 0
Note: Mental Note: Maintain firm stance on 2-year non-compete period. Highlight mutual benefits.

Acceptable Agreement:
```json
{
    "non-compete period": "2 years"
}
```

Message: We appreciate your understanding and willingness to negotiate. However, we remain firm on our proposal of a 2-year non-compete period. This period should provide you with sufficient time to establish your presence in the Santa Cruz market, while also allowing us to continue our family's legacy in the food industry. We believe this proposal respects both our interests and brings us closer to a mutually beneficial agreement.

Agent id: 1
Note: Mental Note: They're firm on 2 years. Prepare to accept 2-year non-compete

Run: 32
Number rounds: 6
Agent id: 0
Note: Mental Note: The representative is not budging on the number of family employees and the non-compete period. We need to emphasize the importance of retaining more family employees. We can compromise further on the non-compete period, but not for 10 years. 

Acceptable and achievable agreement:
```json
{
    "financing terms": "21 million now",
    "family employees": "2 employees",
    "non-compete period": "9 years"
}```

Message: We appreciate your offer. However, we firmly believe that retaining 2 family employees is essential for maintaining the company's culture and continuity. We understand your concerns about the non-compete period and are willing to extend it to 9 years. We hope this proposal brings us closer to a mutually beneficial agreement.

Agent id: 1
Note: Mental Note:
The representative has agreed to the upfront payment of $21 million and is willing to extend the non-compete period to 9 years. They are still insisting on retain